In [4]:
%%capture
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from scipy import integrate
from scipy import interpolate
import math
import sys
from scipy import signal
import pandas as pd
import tensorflow as tf
from numba import cuda
import numba
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from IPython.display import display, HTML
%run KalmanFilter.ipynb
import mpld3
mpld3.enable_notebook()


In [4]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.python.keras import backend as K 
from IPython.display import clear_output
from tensorflow.python.keras.initializers import RandomUniform
# tf.enable_eager_execution()

In [2]:
x_dim = 11
y_dim = 2
def create_input_and_output(data, just_acc=False):
    ## Needed Data
    gps = data.gps[:,1:3] 
    acc = data.acc_with_grav_ERC[:, 1:4]
    mag = data.mag[:, 1:3]
    gyro = data.gyro[:, 1:4]

    time_series = data.acc_with_grav_ERC[:, 0]
    ground_truth = data.ground_truth.dis[:, 1:3]
    delta_time = np.diff(time_series, axis=0)
    delta_time = np.concatenate(([[0]], delta_time))
    
    
    # Choose which data to include in input
    if (just_acc):
        input_data = np.concatenate((gps, acc, delta_time), axis=1)
    else:
        input_data = np.concatenate((gps, acc, gyro, mag, delta_time), axis=1) ## Feature Vector Length = 11
    return input_data, ground_truth


In [4]:
def load_datasets(files):
    training_datasets = []
    for file in files:
        data = Data_Stream(file, load_truth=True, higher_freq=True)
        input_data, ground_truth = create_input_and_output(data)
        training_datasets.append([input_data, ground_truth])
    return training_datasets
        

In [5]:
gps_bound = 3000.0
acc_bound = 3000.0 #Actual bound (30)
gyro_bound = 3000.0 #Actual bound (2)
mag_bound = 3000.0 #Actual bound (30)
dt_bound = 3000.0 #Actual bound (0.1)

custom_scale_matrix = np.asmatrix([gps_bound, gps_bound,
                                   acc_bound, acc_bound, acc_bound, 
                                   gyro_bound, gyro_bound, gyro_bound,
                                   mag_bound, mag_bound, dt_bound])
custom_scale_matrix = np.concatenate((custom_scale_matrix, -custom_scale_matrix))

print(custom_scale_matrix)
x_scaler = MinMaxScaler()
x_scaler = x_scaler.fit(custom_scale_matrix)
y_scaler = MinMaxScaler()
y_scaler = y_scaler.fit(custom_scale_matrix[:, 0:2])

def scale_dataset(training_dataset, test_dataset):
    scaled_training_dataset = []
    scaled_test_dataset = []
    
    for activity in training_dataset:
        scaled_training_dataset.append([x_scaler.transform(activity[0]), 
                                        y_scaler.transform(activity[1])])
    for activity in test_dataset:
        scaled_test_dataset.append([x_scaler.transform(activity[0]), 
                                    y_scaler.transform(activity[1])])
    
    return scaled_training_dataset, scaled_test_dataset
    

[[ 3000.  3000.  3000.  3000.  3000.  3000.  3000.  3000.  3000.  3000.
   3000.]
 [-3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000. -3000.
  -3000.]]


In [6]:
def get_seqs(sequence_length, dataset, offset):
    
    x_seqs = []
    y_seqs = []
    for activity in dataset:
        ##Create all sequences
        for i in range(0, len(activity[0]) - sequence_length, offset):
            x_seqs.append(activity[0][i:i+sequence_length])
            y_seqs.append(activity[1][i:i+sequence_length])
            
    x_seqs = np.asarray(x_seqs)
    y_seqs = np.asarray(y_seqs)
    return x_seqs, y_seqs

def batch_generator(batch_size, x_seqs, y_seqs):  
    
    x_seqs, y_seqs = shuffle(x_seqs, y_seqs)
    #Print number of batches required to pass all sequences in an epoch
    print("Batches per Epoch: ", int(len(x_seqs)/batch_size + 1))
    while True:
        
        #For each batch in an epoch, given n sequences that are in shuffled order
        for i in range(int(len(x_seqs)/batch_size + 1)):
            i1 = i+1
            yield (x_seqs[i*batch_size:i1*batch_size], y_seqs[i*batch_size:i1*batch_size])
        #Shuffle sequences between epochs
        x_seqs, y_seqs = shuffle(x_seqs, y_seqs)
        

In [7]:
def custom_loss(y_true, y_pred):
    
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]
    
    
#     ts = tf.reshape(y_true_slice, (-1, 2))
    
#     ps = tf.reshape(y_pred_slice, (-1, 2))
    
#     def get_min_dist(pi):
# #         print(pi.get_shape(), ts.get_shape())
#         eu_dists = tf.norm(ts-pi, ord='euclidean', axis=1)
#         min_dist = tf.reduce_min(eu_dists)
#         return min_dist


#     min_dists = tf.map_fn(get_min_dist, ps, dtype=tf.float32)
#     print("Minimum distances to Ground Truth Points Shape", min_dists.get_shape())
#     mean_dist = tf.reduce_mean(min_dists)
#     print("Mean Mimimum Distance Shape", mean_dist.get_shape())
    
# #     min_dists_tot = 0.0
# #     print(ps.shape)
# #     for pi in ps:
# #         min_t = sys.float_info.max
# #         for ti in ts:
# #             t = tf.norm(ti-pi, ord='euclidean')
# #             if(t < min_t):
# #                 min_t = t
# #         min_dists_tot += min_t
        
# #     mean_dist = min_dists_tot/len(ps)
    
    eu_dists = tf.norm(y_true_slice-y_pred_slice, ord='euclidean')
    loss_mean = tf.reduce_mean(eu_dists)
    
    return loss_mean

In [8]:
class PlotLosses(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.figure(figsize=(9, 8))
        plt.plot(self.x, self.losses, label="Training Loss", c='r')
        plt.plot(self.x, self.val_losses, label="Validation Loss", c='b')
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, 1.00), ncol=2, frameon=False)
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.ioff()
        plt.show();

In [3]:
def plot_dataset(dataset, seq_len=100):
    
    for activity in dataset:
        input_data = activity[0]
        ground_truth = activity[1]
        orig_gps = activity[0][:, 0:2]
        
        
        #Pad input
        padding = np.zeros((seq_len, input_data.shape[1]))
        input_data = np.concatenate((padding, input_data))

        ##Scale down the trial input, get predicted output
        input_data = x_scaler.transform(input_data)
        input_data = np.expand_dims(input_data, axis=0)
        predicted_output = model.predict(input_data)

        
        ## Remove data upto the start point
        ground_truth = ground_truth[seq_len:]
        orig_gps = orig_gps[seq_len:]
        predicted_output = y_scaler.inverse_transform(predicted_output[0])[seq_len:]

        ##Print Graphs of X against Y
        plt.figure(figsize=(8,5))
        plt.plot(ground_truth[:, 0], ground_truth[:, 1], label='ground truth', color = 'g')
        plt.plot(predicted_output[:, 0], predicted_output[:, 1], label='seen training data', color = 'b')
        plt.plot(orig_gps[:, 1], orig_gps[:, 2], label='original gps', color = 'r')
        plt.legend()
        plt.xlabel('Position X (Metres)')
        plt.ylabel('Position Y (Metres)')
        plt.show()

#         print("Accuracy of GPS: ", measure_accuracy(ground_truth, orig_gps[:, 1:3]))
        print("Accuracy of RNN: ", measure_accuracy(ground_truth, predicted_output))
        

### Print Data with Pandas

In [10]:
# if(print_outputs):

#     if(False):
#         training_frame = pd.DataFrame(np.concatenate((training_data, training_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y', 
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     else:
#         training_frame = pd.DataFrame(np.concatenate((training_data, training_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y',
#     #                                                           'Vel X', 'Vel Y',
#     #                                                           'Mag X', 'Mag Y',
#     #                                                           'Gyro X', 'Gyro Y', 'Gyro Z',
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     print("Training Data")
#     display(training_frame.head(5))
#     training_frame.plot(figsize=(20,10))

In [11]:
# if(print_outputs):

#     if(False):
#         test_frame = pd.DataFrame(np.concatenate((test_data, test_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y', 
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     else:
#         test_frame = pd.DataFrame(np.concatenate((test_data, test_labels), axis=1), 
#                                                      columns=['GPS X', 'GPS Y', 'Acc X', 'Acc Y',
#     #                                                           'Vel X', 'Vel Y',
#     #                                                           'Mag X', 'Mag Y',
#     #                                                           'Gyro X', 'Gyro Y', 'Gyro Z',
#                                                               'Delta Time', 'Ground Truth X', 'Ground_Truth Y'])
#     print("Testing Data")
#     display(test_frame.head(5))
#     # test_frame.plot(figsize=(20,10))
